In [ ]:
##Round 1 cleaning complete. v1 decisions: keep all nan as new feature/value; no skew or outlier action has been taken
##Round 1 exploration complete. (a) Distribution of each feature explored (b) feature-target and feature-feature relationships explored 

##Round 1 modelling Plan: make one model for each notebook, try to do systematic tuning ... take best few models (?) or is it too soon?
##then go back and do feature engineering and feature selection based on these base-line models
##Build pipe-line: dummies, scaling, tuning ...

##next steps:
##
##Flask API Deployment
##build small webpage interaction


### Tuning of KNN 

Main parameters: k = num_neighbors, and metric

In [ ]:
## following https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

In [ ]:
##brief review of KNN vs K-means ... 
##K-means (unspervised) has no labels, just feature space ... you create k-labels (choice of k is tricky and important)
##then run thru k-means clustering algorithm to assign k-labels (clusters) to the data 
##b
##KNN (supervised) is simpler because data already comes with p-labels (the role of k is different here) 
##in the training data on the test data you simply look at the k-nearet neighors (of training set) and take majority vote 
##on what label to assign to new point. The choice of k is important and the metric can change the outcomes.

In [2]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score
import seaborn as sns

import pickle

#sns.set()
sns.set(rc={'figure.figsize':(8,5)})



In [3]:
## cleaning tool


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [36]:
path = "/Users/joejohns/Downloads/data_loans.csv"

data = pd.read_csv(path) 
print(data.shape)
data.head(3)

(614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y


## data processing from round 1


In [5]:
perc_null(data)
    

,Total,Type,Percent
Credit_History,50,float64,0.081433
Self_Employed,32,object,0.052117
LoanAmount,22,float64,0.035831
Dependents,15,object,0.024430
Loan_Amount_Term,14,float64,0.022801
Gender,13,object,0.021173
Married,3,object,0.004886
Loan_ID,0,object,0.000000
Education,0,object,0.000000
ApplicantIncome,0,int64,0.000000


In [45]:
data.rename(columns = {'ApplicantIncome':'Applicant_Income', 'CoapplicantIncome': 'Coapplicant_Income', 'LoanAmount':'Loan_Amount'}, inplace = True)

In [7]:
data.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Applicant_Income', 'Coapplicant_Income',
       'Loan_Amount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status'],
      dtype='object')

In [37]:
target = ['Loan_Status']
ID_feats = ['Loan_ID',] 

#feature types:

num_feats = ['Applicant_Income', 'Coapplicant_Income', 'Loan_Amount', 'Loan_Amount_Term',]

ord_cat_feats = [ 'Dependents', 'Education', ]
nom_cat_feats = ['Gender', 'Married', 'Self_Employed', 'Credit_History', 'Property_Area',]




#create new category values
make_new_catval_unk = [  'Credit_History', 'Self_Employed',  'Dependents', 'Gender', 'Married', ] 

#create new category

make_new_cat_unk = [ 'Loan_Amount', 'Loan_Amount_Term',]

#if any value is unknown for either feature we make it 'yes')
#cat = 'missing_data'; in general we can have several lists of related variables and make a new cat for each list; 
#name of cat should be specified

#imputing

impute_mean_feats = ['Loan_Amount']
impute_freq_feats = [ 'Loan_Amount_Term']


#scaling
scaling_feats = ord_cat_feats+num_feats

#dummies
dummies_feats = nom_cat_feats #default setting of this variable ... 
#ordinal_encode
ord_feats = ord_cat_feats

#rest of categoricals get new catgory value 'unk' ... it is not very predictive in most cases ...

In [38]:


for item in ['0', '1', '2', '3+']:
    if item != '3+':
        i = int(item)
    else:
        i = 3
    data.loc[data['Dependents'] == item,'Dependents']  = i


In [39]:

data.loc[data["Loan_Status"] == 'Y', "Loan_Status"] = 1
data.loc[data["Loan_Status"] == 'N', "Loan_Status"] = 0



In [40]:


for item in ['0', '1', '2', '3+']:
    if item != '3+':
        i = int(item)
    else:
        i = 3
    data.loc[data['Dependents'] == item,'Dependents']  = i
    
#impute_most_freq = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
#data["Dependents"] = impute_most_freq.fit_transform(data["Dependents"])

#gah f-off with your 2D array non-sense ... I'll write my own function


In [41]:

data.loc[data['Credit_History'] ==0.0, 'Credit_History'] = 'No'
data.loc[data['Credit_History'] == 1.0, 'Credit_History'] = 'Yes'  #I use capitalized "Yes", "No" to be conistent with their other entries

In [42]:
data.loc[data['Education']=='Not Graduate', "Education"] = 0
data.loc[data['Education']=='Graduate', "Education"] = 1

In [43]:

#we don't want to do this for cat_ord_feats, because those will be numeric; we do impute_most_freq instead
for cat in nom_cat_feats:
    if data[cat].isnull().sum()>0: 
        data.loc[data[cat].isnull(), cat] = 'unk'

In [46]:
## These are the two numerical features with NaN .. here we set up two new categories to capture these NaN since
## they seem decently predictive. ... Now what to do with the NaN values? I guess just do mean and most-common


#data['Loan_Amount_Term_unk'] = 'No' #set default to 'No' then set a few to be 'Yes'

#create one new categorical for both (if either or both is unknown)
data['missing_data'] = 'No'  
data.loc[(data['Loan_Amount_Term'].isnull()) , 'missing_data'] = 'Yes'
data.loc[(data['Loan_Amount'].isnull()) , 'missing_data'] = 'Yes'
data.loc[(data['Gender'].isnull()) , 'missing_data'] = 'Yes'



In [47]:
#update nom_cat_feats (no null values, ok)

nom_cat_feats+= ['missing_data']

#### Done with cleaning categorical features ... 
note: we have made preliminary decision to make all NaN in categorical columns into new cat_val ... \
we'll see how this works out ... in some cases it seems to be a good predictor.

In phase 2 investigation, below, we will investigate the naive predictive strength of all the \
categorical features (including NaN as 'unk')


In [48]:
##baseline models
from sklearn.model_selection import train_test_split # For train/test splits
from sklearn.neighbors import KNeighborsClassifier # The k-nearest neighbor classifier
from sklearn.feature_selection import VarianceThreshold # Feature selector
from sklearn.pipeline import Pipeline # For setting up pipeline
# Various pre-processing steps
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import GridSearchCV # For optimization


pipeline should do this:

num_prep = Pipeline([
      #  ('keep_num', keep_cat),
        ('most_freq_imp', most_freq_imp),
        ('mean_imp', mean_imp), 
        ])

In [ ]:
##if only one action taken in the pipe-line then don't need it.

#all categoricals (which excludes ordinal cats) will be dummies


#for the two numerical features, one should be most freq, one should be mean 
#most_freq_imp = Pipeline([
      #  ('keep_num', keep_cat),
#        ('most_freq_imp', most_freq_imp),
        
       # ('mean_imp', mean_imp), 
#        ])

#mean_imp = Pipeline([
      #  ('keep_num', keep_cat),
        #('most_freq_imp', most_freq_imp),
#        ('mean_imp', mean_imp), 
#        ])



#cat_dummies = Pipeline([        
       # ('keep_cat', keep_cat),
        #('cat_imp', cat_imp),
#        ('dummies', dummies),
#        ])


In [20]:
data.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Applicant_Income', 'Coapplicant_Income',
       'Loan_Amount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status', 'Loan_info_unk'],
      dtype='object')

target = ['Loan_Status']
ID_feats = ['Loan_ID']
#all features excepy ID and Loan_Status (to be used for modelling)

model_feats= ['Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Applicant_Income', 'Coapplicant_Income',
       'Loan_Amount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area',
        'Loan_Amount_Term_unk', 'Loan_Amount_unk']

##categorical features

'''cat_feats = [ 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed',  'Credit_History', 'Property_Area',
        'Loan_Amount_Term_unk', 'Loan_Amount_unk'  ]

cat_nom_feats = ['Gender', 'Married','Self_Employed',  'Credit_History', 'Property_Area',
        'Loan_Amount_Term_unk', 'Loan_Amount_unk' ]
cat_ord_feats = ['Dependents', 'Education',]

##numerical features 

num_feats = ['Applicant_Income', 'Coapplicant_Income',
       'Loan_Amount', 'Loan_Amount_Term', ]


num_mean_imp_feats = [  'Applicant_Income', 'Coapplicant_Income', 'Loan_Amount']

num_most_freq_imp_feats = ['Loan_Amount_Term',]'''


#3 options for cat_feats with NaN:
#-drop all NaN rows; or a column
#-make NaN into new cat_val = 'unk'
#-impute NaN by most common 
#-impute by most common of k-nearest nbrs

#after NaN dealt with, then dealing with ordinal and nominal cats ... 
#which categories to do dummies and which to do scaling

#cat_ord --> convert to 0,1,2, ... or possibly numbers which have correct SCALE relative to what they are encoding.
cat_scale_feats = cat_ord_feats
cat_dummies_feats = cat_nom_feats 

##numerical features

num_feats = ['Applicant_Income', 'Coapplicant_Income',
       'Loan_Amount', 'Loan_Amount_Term', ]

# options for num_feats with NaN:
#-impute by mean
#-impute NaN by most common 
#-impute by mean of k-nearest nbrs ... 
#-drop all NaN rows or a column
#-make into cat_var by binning; then make NaN into new cat_val = 'unk'
#-add new category with cat_val = 'unk_yes', 'unk_no'; then still have to impute the cts NaN values 


num_mean_imp_feats = [  'Applicant_Income', 'Coapplicant_Income', 'Loan_Amount']
num_most_freq_imp_feats = ['Loan_Amount_Term',]

#scaling

num_scale_feats = num_feats

In [ ]:
##this class under construction ... continue when we get the rest working first

class Nan_unk_Transformer():

    # here you define the operation it should perform
    def transform(self, X, y=None, **fit_params):
        X2 = X.copy()
        X2.loc[X.isnull(), :] = 'unk'
        return X2 

    # just return self
    def fit(self, X, y=None, **fit_params):
        return self

cat_nom_feats = ['Gender', 'Married','Self_Employed',  'Credit_History', 'Property_Area',
        'Loan_Amount_Term_unk', 'Loan_Amount_unk' ]

#we don't want to do this for cat_ord_feats, because those will be numeric; we do impute_most_freq instead
for cat in cat_nom_feats:
    if data[cat].isnull().sum()>0: 
        data.loc[data[cat].isnull(), cat] = 'unk'

#re-doing the cell below ...

###I guess we have to do process1= pipe(...), process2= pipe(...), ...
##then preprocess = columntransformer[(process1, cols1), (process2, cols2) ...]
##processj can be: impute, data compress, feature select, make dummies,  ordnal encoder, scale ... 
##then pipe = pipeline[preprocess, model]

##we will mix up categorical and numerical and simply focus on common processes, eg scaling for ordinal variables 

process: ordinal_encoding (done by hand for now)
process: encoding categoricakl NaN ---> 'unk' (done by hand for now)
process: create new cat for numerical NaN (done by hand for now)

process: impute_most_freq, num_feat ={length of term}
process: impute_mean, num_feats
process: dummies, nom_cats
process: scaling,  num_cats+ord_cats
    
    
    


In [51]:

freq_imp = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
mean_imp = SimpleImputer(missing_values=np.nan, strategy="mean")
scaler = StandardScaler()
dummies = OneHotEncoder(handle_unknown='ignore')

#other options:
#1. create new class that has transform and fit and uses make_dummies instead 
#2. put a FunctionTransformer() in the pipeline 



'''cat_impute_most_freq_feats = cat_ord_feats.copy()
cat_scale_feats = cat_ord_feats.copy()
cat_dummies_feats = cat_nom_feats.copy() 
num_scale_feats = num_feats.copy()
mean_imp_feats = [  'Applicant_Income', 'Coapplicant_Income', 'Loan_Amount']
impute_most_freq_feats = ['Loan_Amount_Term',]

num_mean_imp_feats = [  'Applicant_Income', 'Coapplicant_Income', 'Loan_Amount']
num_impute_most_freq_feats = ['Loan_Amount_Term',]
'''


#imputing

impute_mean_feats = ['Loan_Amount','Gender']
impute_freq_feats = [ 'Loan_Amount_Term']


#scaling
scaling_feats = ord_cat_feats+num_feats

#dummies
dummies_feats = nom_cat_feats #default setting of this variable ... 
#ordinal_encode
ord_feats = ord_cat_feats  #done manually at this point

preprocess = ColumnTransformer([
    ('most_freq_imp', freq_imp, impute_freq_feats),
    ('mean_imp', mean_imp, impute_mean_feats), #only one numerical feature has NaN right now
    ('scaler', scaler, scaling_feats),
    ('dummies', dummies, dummies_feats), 
])

pipeline1 = Pipeline([
    ('preprocess', preprocess),
    ('knn', KNeighborsClassifier())
])

In [26]:

##! Should set up a class which does NaN --> 'unk' imputer method



##set up pipeline; here are the functions we'll use

freq_imp = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
mean_imp = SimpleImputer(missing_values=np.nan, strategy="mean")
scaler = StandardScaler()
dummies = OneHotEncoder(handle_unknown='ignore')


cat_impute_most_freq_feats = ord_cat_feats.copy()
cat_scale_feats = ord_cat_feats.copy()
cat_dummies_feats = nom_cat_feats.copy() 


num_scale_feats = num_feats.copy()

num_mean_imp_feats = [  'Applicant_Income', 'Coapplicant_Income', 'Loan_Amount']
num_impute_most_freq_feats = ['Loan_Amount_Term',]




#no need to impute for categoricals because already did NaN --> 'unk'
#! should turn this into a class and use it's method


categorical_preprocessing = ColumnTransformer([
    ('cat_impute_most_freq', freq_imp, cat_impute_most_freq_feats),
    ('cat_scale', scaler,  cat_scale_feats),
    ('cat_dummies', dummies,  cat_dummies_feats),   
])

numerical_preprocessing = ColumnTransformer([
    ('num_impute_most_freq', freq_imp, num_impute_most_freq_feats ),   #('name', method, [columns])
    ('num_impute_mean', mean_imp,  num_mean_imp_feats),
    ('num_scale', scaler, num_scale_feats)   
])
 

preprocess = Pipeline([
    ('numerical_preprocessing', numerical_preprocessing),   #I switched the order of these two, so dummies is done at the end ... still messes up
    ('categorical_preprocessing', categorical_preprocessing),
    
])
 

#this was my old version ... did cat and num preprocess all mixed up
'''# define which transformer applies to which columns
preprocess = ColumnTransformer([
    ('most_freq_imp', most_freq_imp, num_most_freq_imp),
    ('mean_imp', mean_imp, num_feats), #only one numerical feature has NaN right now
    ('scaler', scaler, num_feats+num_most_freq_imp),
    ('dummies', dummies, cat_feats), 
])'''

pipeline2 = Pipeline([
    ('preprocess', preprocess),
    ('knn', KNeighborsClassifier())
])

In [53]:
#let me now follow https://machinelearningmastery.com/modeling-pipeline-optimization-with-scikit-learn/
#which does cv = 3 instead of cv = stratififedkfold( ... )

model_feats = ord_cat_feats+nom_cat_feats+num_feats

y = data.loc[:, target]
X= data.loc[:, model_feats]

# Split the data into test and train
X_train, X_test, y_train, y_test = train_test_split(
    X,  
    y, 
    test_size=1/3,
    random_state=0)
 
print(X_train.shape)
print(X_test.shape)

#np.array(X)

(409, 12)
(205, 12)


In [54]:
perc_null(data)

,Total,Type,Percent
Loan_Amount,22,float64,0.035831
Dependents,15,object,0.024430
Loan_Amount_Term,14,float64,0.022801
Loan_ID,0,object,0.000000
Gender,0,object,0.000000
Married,0,object,0.000000
Education,0,object,0.000000
Self_Employed,0,object,0.000000
Applicant_Income,0,int64,0.000000
Coapplicant_Income,0,float64,0.000000


In [ ]:
##see stack overflow ... guy who says "Just to add ..."

In [ ]:

knn = pipeline2.fit(X_train, y_train)
print('Training set score: ' + str(knn.score(X_train,y_train)))
print('Test set score: ' + str(knn.score(X_test,y_test)))


In [52]:

knn = pipeline1.fit(X_train, y_train)
print('Training set score: ' + str(knn.score(X_train,y_train)))
print('Test set score: ' + str(knn.score(X_test,y_test)))


ValueError: A given column is not a column of the dataframe

In [ ]:
#This code snipet fails with the error ValueError: Supported target types are: ('binary', 'multiclass'). Got 'unknown' instead.
#Here I am trying to mix these two: 
#just grid search (code below) https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
#pipeline + gridsearch https://machinelearningmastery.com/modeling-pipeline-optimization-with-scikit-learn/

#specifically I am using cv = nice stratified thing from the first one
#in the second one he does test/train split then does grid search with cv =3 


#according to https://chrisalbon.com/code/machine_learning/model_evaluation/cross_validation_pipeline/


#Important note from the scikit docs: For integer/None inputs, if y is binary or multiclass, StratifiedKFold used. 
#    If the estimator is a classifier or if y is neither binary nor multiclass, KFold is used.


# that is, if the estimator is a classifier then kfold and not stratified kfold is used ... ?? wtf ... they 
#should still use stratified kfold ... in fact in principle they should stratify by *all* categorified features ... always


code = '''y = data.loc[:, target]
X= data.loc[:, model_feats]


#model = pipeline
n_neighbors = range(1, 21, 2)
weights = ['uniform', 'distance']
metrics = ['euclidean', 'manhattan', 'minkowski']
# define grid search
grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metrics)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

#grid = GridSearchCV(pipe, parameters, cv=2).fit(X_train, y_train)
#y_train_categorical = keras.utils.to_categorical(y_train, num_classes)


grid_search = GridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)

grid_result = grid_search.fit(np.array(X), np.ravel(y))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))'''

In [ ]:
'''import keras

num_classes = len(set(y))
y_train_categorical = keras.utils.to_categorical(y_train, num_classes)'''

